In [ ]:
import os
from typing import BinaryIO

In [ ]:
f = open('./tokenizer/bpe_example.txt', 'rb')

In [ ]:
f.tell()

In [ ]:
b = f.read(10)
b

In [ ]:
b

In [ ]:
def train_bpe(
        input_path: str | os.PathLike,
        vocab_size: int,
        special_tokens: list[str]
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    path: Path = Path(input_path)
    context: str
    with path.open('+r'):
        
    return {}, []

In [ ]:
p = './tokenizer/bpe_example.txt'

In [ ]:
import os
from pathlib import Path

path: Path = Path(p)
content: str = ''
with path.open('+r') as f:
    content = f.read()
content

In [ ]:
content.encode()

In [ ]:
import regex as re

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

def pretokenizer(content: str, special_tokens: list[str]) -> dict[list[bytes], int]:
    matches = re.finditer(PAT, content)
    counter = {}
    for match in matches:
        text = match.group()
        counter[text] = counter.get(text, 0) + 1
    return counter

In [ ]:
pretokenizer(content, [])

In [ ]:
def remove_special_tokens(content: str, special_tokens: list[str]) -> list[str]:
    pattern = '|'.join(re.escape(d) for d in special_tokens)
    return re.split(pattern, content)


PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""


def pretokenizer(content: str, special_tokens: list[str]) -> dict[list[bytes], int]:
    docs = remove_special_tokens(content, special_tokens)
    counter = {}
    for doc in docs:
        matches = re.finditer(PAT, doc)

        for match in matches:
            text = tuple(match.group().encode('utf8'))
            counter[text] = counter.get(text, 0) + 1
    return counter

In [ ]:
path = Path('data/TinyStoriesV2-GPT4-valid.txt')
with path.open('+r') as f:
    content = f.read()
    print(pretokenizer(content, ['<|endoftext|>']))

In [ ]:
a = 1

In [ ]:
bytes(a)

In [ ]:
for i in range(256):
    print(bytes([i]))

In [ ]:
def init_vocab(special_tokens: list[str]) -> dict[int, bytes]:
    vocab = {}
    for i in range(256):
        vocab[i] = bytes([i])
    for st in special_tokens:
        idx = len(vocab)
        vocab[idx] = st.encode('utf8')
    return vocab

In [ ]:
init_vocab(['<|endoftext|>'])

In [ ]:
def find_max_bp(pretokenized_count: dict[tuple[bytes], int]):
    # Count byte pairs
    bp_counter = {}
    for bytes_tuple, count in pretokenized_count.items():
        for i in range(len(bytes_tuple) - 1):
            bp = (bytes_tuple[i], bytes_tuple[i+1])
            bp_counter[bp] = bp_counter.get(bp, 0) + count
    max_bp, max_count = (bytes([0]), bytes([0])), 0
    for bp, bp_count in bp_counter.items():
        if bp_count > max_count:
            max_bp = bp
            max_count = bp_count
        if bp_count == max_count and bp > max_bp:
            max_bp = bp
            max_count = bp_count
    return max_bp, max_count

In [ ]:
def get_pretokenized_count():
    path = Path('data/TinyStoriesV2-GPT4-valid.txt')
    with path.open('+r') as f:
        content = f.read()
        return pretokenizer(content, ['<|endoftext|>'])

In [ ]:
pretokenized_count = get_pretokenized_count()
max_bp, max_count = find_max_bp(pretokenized_count)

In [ ]:
max_bp

In [ ]:
b1 = b'\x01\x02'

In [ ]:
len(b1)

In [ ]:
bytes(max_bp)

In [ ]:
import pickle

In [ ]:
with open('./output/tinystory_train_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

print(len(vocab))

In [ ]:
with open('./output/tinystory_train_merges.pkl', 'rb') as f:
    merges = pickle.load(f)

print(len(merges))

In [1]:
import torch
from einops import rearrange, einsum

In [4]:
x = torch.arange(120).reshape([2,3,4,5])
x

tensor([[[[  0,   1,   2,   3,   4],
          [  5,   6,   7,   8,   9],
          [ 10,  11,  12,  13,  14],
          [ 15,  16,  17,  18,  19]],

         [[ 20,  21,  22,  23,  24],
          [ 25,  26,  27,  28,  29],
          [ 30,  31,  32,  33,  34],
          [ 35,  36,  37,  38,  39]],

         [[ 40,  41,  42,  43,  44],
          [ 45,  46,  47,  48,  49],
          [ 50,  51,  52,  53,  54],
          [ 55,  56,  57,  58,  59]]],


        [[[ 60,  61,  62,  63,  64],
          [ 65,  66,  67,  68,  69],
          [ 70,  71,  72,  73,  74],
          [ 75,  76,  77,  78,  79]],

         [[ 80,  81,  82,  83,  84],
          [ 85,  86,  87,  88,  89],
          [ 90,  91,  92,  93,  94],
          [ 95,  96,  97,  98,  99]],

         [[100, 101, 102, 103, 104],
          [105, 106, 107, 108, 109],
          [110, 111, 112, 113, 114],
          [115, 116, 117, 118, 119]]]])

In [5]:
x.transpose(0, 2, 3, 1)

TypeError: transpose() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [10]:
rearrange(x, 'b h w c -> b w h c')

tensor([[[[  0,   1,   2,   3,   4],
          [ 20,  21,  22,  23,  24],
          [ 40,  41,  42,  43,  44]],

         [[  5,   6,   7,   8,   9],
          [ 25,  26,  27,  28,  29],
          [ 45,  46,  47,  48,  49]],

         [[ 10,  11,  12,  13,  14],
          [ 30,  31,  32,  33,  34],
          [ 50,  51,  52,  53,  54]],

         [[ 15,  16,  17,  18,  19],
          [ 35,  36,  37,  38,  39],
          [ 55,  56,  57,  58,  59]]],


        [[[ 60,  61,  62,  63,  64],
          [ 80,  81,  82,  83,  84],
          [100, 101, 102, 103, 104]],

         [[ 65,  66,  67,  68,  69],
          [ 85,  86,  87,  88,  89],
          [105, 106, 107, 108, 109]],

         [[ 70,  71,  72,  73,  74],
          [ 90,  91,  92,  93,  94],
          [110, 111, 112, 113, 114]],

         [[ 75,  76,  77,  78,  79],
          [ 95,  96,  97,  98,  99],
          [115, 116, 117, 118, 119]]]])

In [12]:
rearrange(x, 'b h w c -> b (h w) c')

tensor([[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19],
         [ 20,  21,  22,  23,  24],
         [ 25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39],
         [ 40,  41,  42,  43,  44],
         [ 45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54],
         [ 55,  56,  57,  58,  59]],

        [[ 60,  61,  62,  63,  64],
         [ 65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74],
         [ 75,  76,  77,  78,  79],
         [ 80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94],
         [ 95,  96,  97,  98,  99],
         [100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114],
         [115, 116, 117, 118, 119]]])

In [1]:
import torch

In [10]:
x = torch.randn(4, 5)
print(x)
print(x.shape)

tensor([[ 1.0202,  1.1880,  0.3029,  1.0919, -0.4739],
        [-0.4320, -0.4433, -2.1584, -0.0684, -0.7498],
        [-0.3768,  0.4320,  0.0348,  1.3644,  0.8986],
        [ 0.7867,  0.1175, -1.3295,  0.8390,  0.4578]])
torch.Size([4, 5])


In [19]:
m = x.max(dim=1, keepdim=True)
print(m)
print(m.values.shape)

torch.return_types.max(
values=tensor([[ 1.1880],
        [-0.0684],
        [ 1.3644],
        [ 0.8390]]),
indices=tensor([[1],
        [3],
        [3],
        [3]]))
torch.Size([4, 1])


In [21]:
x.select(1)

TypeError: select() received an invalid combination of arguments - got (int), but expected one of:
 * (name dim, int index)
 * (int dim, int index)
